# BigQuery - BQML Online Predictions

This notebook exports the BigQuery ML model created in `02 - BigQuery - BQML` and then uses Vertex AI to upload the model and deploy it to an endpoint for online predictions.

**Prerequisites**
- `00 - Initial Setup`
- `01 - BigQuery - Data`
- `02 - BigQuery - BQML`

**Overview**

<img src="architectures/statmike-mlops-03.png">

---
## Setup

Setup Parameters For Model Deployment

In [4]:
PROJECT_ID='statmike-mlops'
REGION='us-central1'

MODEL_NAME='MODEL_03_BQML-DIGITS'
ENDPOINT_NAME='ENDPOINT_03_BQML-DIGITS'
MODEL_DIR='gs://{}/digits/model/03_bqml'.format(PROJECT_ID)
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

params = {"MODEL_DIR":MODEL_DIR}
DEPLOY_IMAGE='us-docker.pkg.dev/cloud-aiplatform/prediction/tf2-cpu.2-2:latest'
DEPLOY_COMPUTE='n1-standard-4'

Setup AI Platform Python Clients
- https://googleapis.dev/python/aiplatform/latest/index.html

In [2]:
from google.cloud import aiplatform

API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)
client_options = {"api_endpoint": API_ENDPOINT}
clients = {}

---
## Export the BigQuery Model

Export the BigQuery Model:
- https://cloud.google.com/bigquery-ml/docs/exporting-models

In [3]:
%%bigquery --params $params
EXPORT MODEL `digits.digits_lr`
OPTIONS(URI = @MODEL_DIR)

Query complete after 0.01s: 100%|██████████| 2/2 [00:00<00:00, 854.76query/s]                         


""


---
## Upload the Model to Vertex AI

Import the Model:
- https://cloud.google.com/ai-platform-unified/docs/predictions/importing-custom-trained-model

Create a client to the Model Service, define the Model, and upload the model:

In [5]:
clients['model'] = aiplatform.gapic.ModelServiceClient(client_options=client_options)

MODEL = {
    "display_name": MODEL_NAME,
    "metadata_schema_uri": "",
    "artifact_uri": MODEL_DIR,
    "container_spec": {
        "image_uri": DEPLOY_IMAGE,
        "command": [],
        "args": [],
        "env": [],
        "ports": [{"container_port": 8080}],
        "predict_route": "",
        "health_route": ""
    }
}

uploaded_model = clients['model'].upload_model(parent=PARENT, model=MODEL)

Retrieve the model information and view the name and display name:

In [6]:
model_info = clients['model'].get_model(name=uploaded_model.result(timeout=180).model)
model_info.display_name, model_info.name

('MODEL_03_BQML-DIGITS',
 'projects/691911073727/locations/us-central1/models/7528197785707872256')

---
## Create the Vertex AI Endpoint

Deploy the Model:
- https://cloud.google.com/ai-platform-unified/docs/predictions/deploy-model-api

Create a client to the Endpoint Service and use it to create the endpoint:

In [7]:
clients['endpoint'] = aiplatform.gapic.EndpointServiceClient(client_options=client_options)

endpoint = clients['endpoint'].create_endpoint(parent=PARENT, endpoint={"display_name": ENDPOINT_NAME})

Retrieve the endpoint information and view the name and display name:

In [8]:
endpoint_info = clients['endpoint'].get_endpoint(name=endpoint.result(timeout=180).name)
endpoint_info.display_name, endpoint_info.name

('ENDPOINT_03_BQML-DIGITS',
 'projects/691911073727/locations/us-central1/endpoints/1208284114088624128')

---
## Deploy the Model to the Vertex AI Endpoint

In [9]:
DMODEL = {
        "model": model_info.name,
        "display_name": 'DEPLOYED_'+MODEL_NAME,
        "dedicated_resources": {
            "min_replica_count": 1,
            "max_replica_count": 1,
            "machine_spec": {
                    "machine_type": DEPLOY_COMPUTE,
                    "accelerator_count": 0,
                }
        }   
}

TRAFFIC = {
    '0' : 100
}

dmodel = clients['endpoint'].deploy_model(endpoint=endpoint_info.name, deployed_model=DMODEL, traffic_split=TRAFFIC)

Retrieve the deployed model information from the endpoint:

In [16]:
clients['endpoint'].get_endpoint(name=endpoint_info.name).deployed_models

[id: "7706204320198295552"
model: "projects/691911073727/locations/us-central1/models/7528197785707872256"
display_name: "DEPLOYED_MODEL_03_BQML-DIGITS"
create_time {
  seconds: 1625579068
  nanos: 450381000
}
dedicated_resources {
  machine_spec {
    machine_type: "n1-standard-4"
  }
  min_replica_count: 1
  max_replica_count: 1
}
]

---
## Predictions

Get an online prediction:
- https://cloud.google.com/ai-platform-unified/docs/predictions/online-predictions-custom-models#online_predict_custom_trained-python

Create a client to the prediction service:

In [17]:
clients['prediction'] = aiplatform.gapic.PredictionServiceClient(client_options=client_options)

Setup an observation for prediction:

In [18]:
%%bigquery pred
SELECT *
FROM `digits.digits_prepped`
WHERE splits='TEST'

Downloading: 100%|██████████| 366/366 [00:01<00:00, 330.88rows/s]


In [19]:
pred.head(1)

,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9,...,p57,p58,p59,p60,p61,p62,p63,target,target_OE,SPLITS
0,0.0,0.0,0.0,9.0,8.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,7.0,14.0,9.0,0.0,0.0,0,Even,TEST


In [20]:
newob = pred.loc[:0,'p0':'p63'].to_dict(orient='records')[0]
#newob

Request prediction from the prediction service:

In [21]:
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value

response = clients['prediction'].predict(endpoint=endpoint_info.name, instances=[json_format.ParseDict(newob, Value())], parameters=json_format.ParseDict({}, Value()))

Review the prediction response (note the response is in protobuf format):

In [22]:
#response.predictions._pb

In [23]:
response.predictions._pb[0].struct_value.fields['predicted_target'].list_value.values[0].string_value

'0'

---
## Remove Resources
- undeploy-model
- remove endpoint
- remove model
- delete model files

Undeploy Model:

In [233]:
dmodel = clients['endpoint'].get_endpoint(name=endpoint_info.name).deployed_models[0].id
clients['endpoint'].undeploy_model(endpoint=endpoint_info.name, deployed_model_id=dmodel)

Delete Endpoint:

In [234]:
clients['endpoint'].delete_endpoint(name=endpoint_info.name)

Remove Model:

In [235]:
clients['model'].delete_model(name=model_info.name)

Delete Model Files:

In [236]:
from google.cloud import storage
gcs = storage.Client()

path = gcs.bucket(PROJECT_ID)
blobs = path.list_blobs(prefix='digits/bqml')
for blob in blobs:
    blob.delete()